# Plasma Line Formulation
Create a plasma line dataset with SimISR.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from ISRSpectrum import PLspecinit


import scipy.signal as sig
import scipy.constants as sconst
import xarray as xr
import pandas as pd

from SimISR import MakePulseDataRepLPC,make_pline_specds
from SimISR.testfunctions import temp_profile, chapman_func

In [2]:
xy = np.zeros(1)
z_in = np.linspace(100,1000,100)
xm, ym, zm = np.meshgrid(xy, xy, z_in)

z_in = zm.flatten()
N_0=1e12
z_0=250.0
H_0=50.0
Ne_profile = chapman_func(z_in, H_0, z_0, N_0)
(Te_prof, Ti_prof) = temp_profile(z_in)

Te = Te_prof[:, np.newaxis].repeat(2, axis=1)
Ti = Te_prof[:, np.newaxis].repeat(2, axis=1)

times = pd.date_range("2024-01-01T15:24:00Z", freq=pd.Timedelta(hours=1), periods=2)
d1 = ["locs", "time"]
# d2 = ['locs','time','spdims']

Ne_all = Ne_profile[..., np.newaxis].repeat(2, axis=1)
Ni_non = np.zeros_like(Ne_all)
idict = {
    "ne": (d1, Ne_all, {"units": "m^-3"}),
    "nO+": (d1, Ne_all, {"units": "m^-3"}),
    "nH+": (d1, Ni_non, {"units": "m^-3"}),
    "nHe+": (d1, Ni_non, {"units": "m^-3"}),
    "nO2+": (d1, Ni_non, {"units": "m^-3"}),
    "nN0+": (d1, Ni_non, {"units": "m^-3"}),
    "nN+": (d1, Ni_non, {"units": "m^-3"}),
    "Te": (d1, Te, {"units": "K"}),
    "Ti": (d1, Ti, {"units": "K"}),
    "vdop": (d1, Ni_non, {"units": "m/s"}),
}

coords = dict(
    x=(["locs"], xm.flatten(), {"units": "km"}),
    y=(["locs"], ym.flatten(), {"units": "km"}),
    z=(["locs"], zm.flatten(), {"units": "km"}),
    time=times,
)
# spdims=[0,1,2])

attrs = dict(originlla=np.array([42.6195, -71.49173, 146.0]), species=["O+", "e-"])
i_ds = xr.Dataset(idict, coords=coords, attrs=attrs)

In [3]:

fs = 25e6
nfreqchans = 250
nfreq_pfb = 1024
nfreqall = nfreqchans * nfreq_pfb
# create the PLspec object
spec_args = dict(centerFrequency=440.2 * 1e6,
    bMag=0.4e-4,
    dFlag=False,
    fs=fs,
    nchans=nfreqchans,
    nfreq_pfb=nfreq_pfb,
)
sp_ds = make_pline_specds(i_ds,spec_args)





In [ ]:
# pls = PLspecinit(**spec_args)
# #     centerFrequency=440.2 * 1e6,
# #     bMag=0.4e-4,
# #     dFlag=False,
# #     fs=fs,
# #     nchans=nfreqchans,
# #     nfreq_pfb=nfreq_pfb,
# # )


# attrs = i_ds.attrs.copy()
# coords = i_ds.coords.copy()
# idx_coords = coords.copy().to_index()

# coords['freq_chans'] = pls.freq_vec
# coords['freq_center'] = pls.cfreqvec


# ne= i_ds.ne.to_numpy()
# nloc,ntime = ne.shape

# dims = ['locs','time','freq_chans','freq_center']


# sr_num = int(fs)
# sr_den = 1
# attrs['sr_num'] = sr_num
# attrs['sr_den'] = sr_den
# rc_arr = np.empty((nloc,ntime))
# he_arr = np.empty((nloc,ntime))
# #Hack may have to change the number representation of 
# sp_arr = np.empty((nloc,ntime,spec_args['nchans'],spec_args['nfreq_pfb']))
# d_dict = {'pline':(dims, sp_arr), 'rcs':(dims[:-2], rc_arr), 'he':(dims[:-2], he_arr)}
# sp_ds = xr.Dataset(d_dict,coords=coords,attrs=attrs)

# species = attrs['species']
# klist = list(i_ds.keys())
# den_names = ['ne']*len(species)
# t_names = ['Ti']*len(species)
# t_names[-1] = 'Te'

# for inum,isp in enumerate(species[:-1]):
#     den_names[inum] = 'n'+isp
#     if 't'+isp in klist:
#         t_names[inum] = 't'+isp

# data_block = np.empty((2, 2))

# for idx in idx_coords:
#     data_block[0,0] = i_ds["ne"].loc[idx].data
#     data_block[-1, 0] = i_ds["ne"].loc[idx].data
#     data_block[-1, 1] = i_ds["Te"].loc[idx].data
#     sum_den = 0.
#     sum_t = 0.
#     for inum, (iden, itemp) in enumerate(zip(den_names, t_names)):
#         cur_n =i_ds[iden].loc[idx].data
#         sum_den+=cur_n
#         sum_t += i_ds[itemp].loc[idx].data * cur_n
#     data_block[1, 1] = sum_t/sum_den
#     pld = pls.get_ul_spec(
#     data_block, 0.0, rcsflag=True, heflag=True, freqflag=True, chanflag = True)

#     (f_lo, sp_lo, f_hi, sp_hi, rcs, he, freqs, chan_l, chan_h) = pld
#     sp_ds.rcs.loc[idx] = rcs
#     sp_ds.he.loc[idx] = he * 1e-3
#     for isplo, isphi, ichl, ichh in zip(sp_lo, sp_hi, chan_l, chan_h):

#         sp_ds.pline.loc[idx][ichl]= isplo
#         sp_ds.pline.loc[idx][ichh]= isphi
#     pl_cf[inum] = freqs
    